In [1]:
import json
import torch
from datasets import Dataset

class QnADataset :
    def __init__(self, data_path, tokenizer, max_length=512) :
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.data = self.load_data(data_path)

    def load_data(self, data_path) :
        with open(data_path, 'r', encoding="utf-8") as f :
            data = json.load(f)
        return data
    
    def prepare_input_output(self, item) :
        input_text = f"질문 : {item['question']}\n문서 : {item['context']}\m답변 : "

        output_text = item["answer"]
        
        return input_text, output_text
    
    def tokenize_data(self) :
        input_ids_list = []
        attention_mask_list = []
        labels_list = []

        for item in self.data :
            input_text, output_text = self.prepare_input_output(item)

            input_tokens = self.tokenizer(
                input_text,
                add_special_tokens=False,
                return_tensors="pt"
            )["input_ids"].squeeze()

            output_tokens = self.tokenizer(
                output_text,
                add_special_tokens=False,
                return_tensors="pt"
            )["input_ids"].squeeze()

            full_sequence = torch.cat([input_tokens, output_tokens])

            labels = torch.cat([
                torch.full_like(input_tokens, -100),
                output_tokens
            ])

            if len(full_sequence) > self.max_length :
                full_sequence = full_sequence[:self.max_length]
                labels = labels[:self.max_length]

            attention_mask = torch.ones_like(full_sequence)

            input_ids_list.append(full_sequence)
            attention_mask_list.append(attention_mask)
            labels_list.append(labels)

        input_ids_list = torch.nn.utils.rnn.pad_sequence(
            input_ids_list, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )

        attention_mask_list = torch.nn.utils.rnn.pad_sequence(
            attention_mask_list, batch_first=True, padding_value=0
        )

        labels_list = torch.nn.utils.rnn.pad_sequence(
            labels_list, batch_first=True, padding_value=-100
        )

        dataset = Dataset.from_dict({
            "input_ids" : input_ids_list,
            "attention_mask" : attention_mask_list,
            "labels" : labels_list
        })

        return dataset

In [2]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer

def setup_model_and_tokenizer(model_name) :
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map={"":0},
        trust_remote_code=True
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side="right"

    return model, tokenizer

In [3]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
def setup_lora_config() :
    lora_config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM"
    )

    return lora_config

In [4]:
def create_sample_data() :
    sample_data = [
        {
            "question": "파이썬에서 리스트를 어떻게 정렬하나요?",
            "context": "파이썬 리스트는 sort() 메서드나 sorted() 함수를 사용하여 정렬할 수 있습니다. sort()는 원본 리스트를 수정하고, sorted()는 새로운 정렬된 리스트를 반환합니다.",
            "answer": "파이썬에서 리스트를 정렬하는 방법은 두 가지입니다. 1) list.sort() - 원본 리스트를 직접 수정하여 정렬합니다. 2) sorted(list) - 원본을 유지하고 새로운 정렬된 리스트를 반환합니다."
        },
        {
            "question": "딥러닝에서 과적합이란 무엇인가요?",
            "context": "과적합(Overfitting)은 모델이 훈련 데이터에 너무 특화되어 새로운 데이터에 대한 일반화 성능이 떨어지는 현상입니다. 훈련 정확도는 높지만 검증 정확도가 낮은 특징을 보입니다.",
            "answer": "과적합은 모델이 훈련 데이터에만 과도하게 맞춰져서 새로운 데이터에 대한 예측 성능이 떨어지는 현상입니다. 드롭아웃, 정규화, 조기 종료 등의 방법으로 방지할 수 있습니다."
        }
    ]

    with open("./data/qna_data.json", 'w', encoding="utf-8") as f :
        json.dump(sample_data, f, ensure_ascii=False, indent=2)

    print("샘플 데이터 생성")

In [5]:
from transformers import TrainingArguments, DataCollatorForLanguageModeling, Trainer

def main() :
    model_name = "./model/LLM/deepseek-qwen-bllossom-32b"
    data_path = "./data/qna_data.json"
    output_dir = "./model/finetuned-model"

    model, tokenizer = setup_model_and_tokenizer(model_name)

    model = prepare_model_for_kbit_training(model)

    lora_config = setup_lora_config()
    model = get_peft_model(model, lora_config)

    dataset_handler = QnADataset(data_path, tokenizer)
    train_dataset = dataset_handler.tokenize_data()

    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        save_strategy="epoch",
        eval_strategy="no",
        warmup_steps=100,
        lr_scheduler_type="cosine",
        remove_unused_columns=False,
        dataloader_pin_memory=False
        # dataloader_num_workers=0,
        # ddp_find_unused_parameters=False
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer
    )

    print("파인튜닝 시작")
    trainer.train()

    trainer.save_model()
    tokenizer.save_pretrained(output_dir)

    print(f"파인튜닝 완료 : {output_dir}")

In [ ]:
if __name__ == "__main__" :
    create_sample_data()
    main()

샘플 데이터 생성


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

/tmp/ipykernel_361568/43731301.py:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


파인튜닝 시작


/opt/conda/envs/sangwon/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


/opt/conda/envs/sangwon/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/envs/sangwon/lib/python3.11/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


파인튜닝 완료 : ./model/finetuned-model


: 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

def load_finetuned_model(base_model_path, finetuned_model_path):
   base_model = AutoModelForCausalLM.from_pretrained(
       base_model_path,
       torch_dtype=torch.float16,
       device_map="auto",
       trust_remote_code=True
   )
   
   tokenizer = AutoTokenizer.from_pretrained(finetuned_model_path)
   
   model = PeftModel.from_pretrained(base_model, finetuned_model_path)
   model = model.merge_and_unload() 
   
   return model, tokenizer

def generate_answer(model, tokenizer, question, context, max_length=512):
   input_text = f"질문 : {question}\n문서 : {context}\n답변 : "
   
   inputs = tokenizer(
       input_text,
       return_tensors="pt",
       truncation=True,
       max_length=max_length
   ).to(model.device)
   
   with torch.no_grad():
       outputs = model.generate(
           **inputs,
           max_new_tokens=200,
           temperature=0.7,
           do_sample=True,
           pad_token_id=tokenizer.eos_token_id,
           eos_token_id=tokenizer.eos_token_id
       )
   
   full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
   
   answer = full_response.replace(input_text, "").strip()
   
   return answer

def main():
   base_model_path = "./model/LLM/deepseek-qwen-bllossom-32b"
   finetuned_model_path = "./model/finetuned-model"
   
   model, tokenizer = load_finetuned_model(base_model_path, finetuned_model_path)
   
   test_cases = [
       {
           "question": "파이썬에서 리스트를 어떻게 정렬하나요?",
           "context": "파이썬 리스트는 sort() 메서드나 sorted() 함수를 사용하여 정렬할 수 있습니다. sort()는 원본 리스트를 수정하고, sorted()는 새로운 정렬된 리스트를 반환합니다."
       },
       {
           "question": "딥러닝에서 과적합을 방지하는 방법은?",
           "context": "과적합 방지 기법으로는 드롭아웃, L1/L2 정규화, 조기 종료, 데이터 증강, 배치 정규화 등이 있습니다. 이러한 기법들은 모델의 일반화 성능을 향상시킵니다."
       }
   ]
   
   for i, test_case in enumerate(test_cases, 1):
       print(f"\n=== 테스트 {i} ===")
       print(f"질문: {test_case['question']}")
       print(f"문서: {test_case['context']}")
       
       answer = generate_answer(
           model, 
           tokenizer, 
           test_case['question'], 
           test_case['context']
       )
       
       print(f"답변: {answer}")
       print("-" * 50)

if __name__ == "__main__":
   main()